In [1]:
import pandas as pd
import os
import pickle

In [21]:
#Download the initial test statistics
df = pd.read_csv('Data/permutation_test_files_metabolomics/initial_tstats.csv', index_col=0)
#df = pd.read_csv('Data/permutation_test_files_proteomics/initial_tstats.csv', index_col=0)


In [22]:
path = os.getcwd() + '\\Data\\permutation_test_files_metabolomics'
#path = os.getcwd() + '\\Data\\permutation_test_files_proteomics'

In [24]:
final_list = []

for filename in os.listdir(path):
    if filename.startswith('Run'):
        file_num = int(filename[3:-4])
        if file_num < 6:
            print(filename)
            with open(os.path.join(path, filename), 'rb') as f: # open in readonly mode
                list1 = pickle.load(f)
            final_list.append(list1)

Run1.txt
Run2.txt
Run3.txt
Run4.txt
Run5.txt


In [26]:
len(final_list)

for list1 in final_list:
    print(len(list1))
    print(list1[:5])

10296
[0.24337239352125428, -0.011366924360905563, 0.22336974083843036, 0.21587802738661835, 0.04949002003751546]
10296
[-0.20153733476360788, -0.059125113507662995, 0.22031714680168507, -0.2326411715202603, 0.04158696037794796]
10296
[-0.017703976786846466, -0.049830863967023784, -0.00012125260924621696, -0.05798569430903941, 0.12171904330864303]
10296
[-0.041282081323160726, -0.031020826682690322, 0.008708389589076349, -0.08256747942164831, -0.017821865013440852]
10296
[-0.014450872542700749, 0.017298043624769815, 0.31496255706020443, -0.01977362132237445, -0.02907153705181964]


In [14]:
sig_edges = []
edgelist = df.index

for index,pathway_pair in enumerate(edgelist):   #test all pathways
    comparison = df.Initial_tstat[index]    #get initial test statistic
    counter = 0
    
    for list1 in final_list:  #len(final_list) = number of permutations
        if abs(list1[index]) >= comparison:   
            counter += 1
    
    p_val = (counter/len(final_list))    #divide number of tests above or equal to the test statistic by total number of tests

    if p_val < 0.01:  #for 100,000 permutations
        sig_edges.append(pathway_pair)  
        
print(len(sig_edges))

95354


### Another alternative: (but converting to numpy array actually seems to take up more memory)

In [44]:
import numpy as np

final_matrix = np.empty((0,len(df)), int)

for filename in os.listdir(path):
    if filename != 'initial_tstats.csv':
        with open(os.path.join(path, filename), 'rb') as f: # open in readonly mode
            list1 = pickle.load(f)
        final_matrix = np.append(final_matrix, np.array([list1]), axis=0)

In [36]:
final_matrix.shape

(10, 10296)

In [45]:
final_matrix = np.array(final_list) 

final_matrix.shape

sig_edges = []
edgelist = df.index

for index,pathway_pair in enumerate(edgelist):   #test all pathways
    comparison = df.Initial_tstat[index]    #get initial test statistic
    counter = 0
    
    col = abs(final_matrix[:,index])  #index a column and turn into absolute values
    counter = (col >= comparison).sum()

    
    p_val = (counter/len(final_list))    #divide number of tests above or equal to the test statistic by total number of tests

    if p_val < 0.01:  #for 100,000 permutations
        sig_edges.append(pathway_pair)  
        
print(len(sig_edges))

1316


In [46]:
from sys import getsizeof
print(getsizeof(final_list))
print(getsizeof(final_matrix))

184
823808


In [35]:

with open ('metabolomics/Data/sig_edges.txt', 'w') as file:
     file.write(','.join(str(i) for i in sig_edges))

-0.5646745805638236


### Read in significant edges files and compare the results

In [20]:
with open('Data/permutation_test_files_metabolomics/sig_edges_100kpermutations.txt') as f:
    lines = f.readlines()

sig_list = lines[0].split(';')
print(len(sig_list))
print(sig_list[0])


#252
#282
#268
#258 out of 10,296 edges significant at p < 0.01


258
R-HSA-1483257, R-HSA-1368082
